In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

from fastai import *
from fastai.vision import *
from ipyexperiments import *
from fastai.basic_train import *
import torchvision

Maybe use `loss(reduction="sum")` and average before `real_step()`

### MNIST batch size = 32, no accumulation

`effective batch size = 32`

In [22]:
import sys
sys.path.append("../dev")
from data_utils import seed_everything
from fastai.train import *

In [23]:
# seed everything for reproducibility
seed_everything(42)

In [24]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path, bs=32)

In [80]:
learn = create_cnn(data, torchvision.models.vgg11, metrics=accuracy)

In [83]:
learn = create_cnn(data, torchvision.models.vgg11, metrics=accuracy,
                   custom_head=nn.Sequential(*([AdaptiveConcatPool2d(), Flatten()]
                                             + bn_drop_lin(1024, 512, bn=False, p=0.5, actn=nn.ReLU())
                                             + bn_drop_lin(512, 2, bn=False, p=0.5))))

In [84]:
learn.model

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace)
      (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): ReLU(inplace)
      (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (14): ReLU(inplace)
      (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

In [85]:
learn.loss_func = CrossEntropyFlat(reduction='mean')

In [86]:
learn.fit(4)

epoch,train_loss,valid_loss,accuracy
1,0.198978,0.105300,0.969578
2,0.298119,0.101548,0.970069
3,0.137865,0.084628,0.978410
4,0.102842,0.072043,0.978901


### MNIST batch size=4, accumulate every n_step=8, 4 epochs

`effective batch size = 32 (bs x n_step)`

In [87]:
# seed everything for reproducibility
seed_everything(42)

In [88]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path, bs=2)

In [95]:
learn = create_cnn(data, torchvision.models.vgg11, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=16)],
                   custom_head=nn.Sequential(*([AdaptiveConcatPool2d(), Flatten()]
                                             + bn_drop_lin(1024, 512, bn=False, p=0.5, actn=nn.ReLU())
                                             + bn_drop_lin(512, 2, bn=False, p=0.5))))

In [96]:
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [97]:
learn.fit(4)

epoch,train_loss,valid_loss,accuracy
1,0.247387,0.123647,0.978410
2,0.145434,0.101106,0.979392
3,0.230068,0.116149,0.977429
4,0.075402,0.116603,0.977429
